In [4]:
import pandas as pd
import html5lib
import requests
from bs4 import BeautifulSoup
import time
import urllib.request  # we are going to need to generate a Request object

In [7]:
def transfermarkt_call(url):
    # here we define the headers for the request
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    # this request object will integrate your URL and the headers defined above
    req = urllib.request.Request(url=url, headers=headers)

    # calling urlopen this way will automatically handle closing the request
    with urllib.request.urlopen(req) as response:
        page_html = response.read()
    return page_html

# Scrape Teams

In [8]:
super_league = "https://www.transfermarkt.com/raiffeisen-super-league/startseite/wettbewerb/C1"
html = transfermarkt_call(super_league)

In [9]:
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table',{"class":"items"})

In [13]:
str(soup.find_all('table',{"class":"items"})[0])

'<table class="items">\n<thead>\n<tr>\n<th colspan="2" id="yw1_c0">Club</th><th class="hide" id="yw1_c1"><a class="sort-link" href="/raiffeisen-super-league/startseite/wettbewerb/C1/sort/name">name</a></th><th class="hide" id="yw1_c2"><a class="sort-link" href="/raiffeisen-super-league/startseite/wettbewerb/C1/sort/name">name</a></th><th class="zentriert" id="yw1_c3"><a class="sort-link" href="/raiffeisen-super-league/startseite/wettbewerb/C1/sort/anzahl_spieler_hidden.desc">Squad</a></th><th class="zentriert hide-for-small hide-for-pad" id="yw1_c4"><a class="sort-link" href="/raiffeisen-super-league/startseite/wettbewerb/C1/sort/alter_durchschnitt.desc">ø age</a></th><th class="zentriert hide-for-pad hide-for-small" id="yw1_c5"><a class="sort-link" href="/raiffeisen-super-league/startseite/wettbewerb/C1/sort/legionaere.desc">Players playing abroad</a></th><th class="rechts hide-for-small hide-for-pad" id="yw1_c6"><a class="sort-link" href="/raiffeisen-super-league/startseite/wettbewer

In [50]:
df = pd.DataFrame(pd.read_html(str(soup.find_all('table',{"class":"items"})[0]))[0])
new_columns = df.columns[2:]
new_columns = new_columns.drop('name.1')
#df = df[new_columns]
#df.drop(df.columns[0])
df = df.drop(df.columns[:1], 1)
df = df.drop(df.columns[1], 1)
df = df.drop(df.columns[-1], 1)
df.columns = new_columns
df['ø age'] = df['ø age']/10
df.to_csv('../data/teams.csv')


0     25.2
1     23.9
2     24.9
3     24.6
4     24.1
5     25.9
6     24.9
7     23.6
8     25.6
9     26.4
10    27.7
Name: ø age, dtype: float64

In [21]:
df

Index(['Club.1', 'name', 'name.1', 'Squad', 'ø age', 'Players playing abroad',
       'Total market value', 'ø market value', 'Total MV', 'ø MV'],
      dtype='object')

In [252]:

my_url = "https://www.transfermarkt.com/bsc-young-boys/kader/verein/452/saison_id/2019"

In [325]:
#page = requests.get(my_url)
soup = BeautifulSoup(page_html, 'html.parser')
table = soup.find_all('table',{"class":"items"})
#table_header = table[0].theader
table = table[0].tbody
player_rows = table.find_all("tr")

In [ ]:
https://stackoverflow.com/questions/55992681/how-to-scrape-a-website-table-where-the-cell-values-have-the-same-class-name
all_tr = soup.find_all('tr', {'class': ['odd', 'even']})
print('rows:', len(all_tr))

for row in all_tr:
    all_td = row.find_all('td', recursive=False)

    print('columns:', len(all_td))
    for column in all_td:
        print(' >', column.text)

    data['name'].append( all_td[1].text.split('.')[0][:-1] )
    data['data of birth'].append( all_td[2].text[:-5])
    data['height'].append( all_td[4].text )
    data['foot'].append( all_td[5].text )
    data['joined'].append( all_td[6].text )
    data['contract until'].append( all_td[8].text )


df = pd.DataFrame(data)
print(df.head())

In [324]:
table_header

In [249]:
player_rows[]

SyntaxError: invalid syntax (<ipython-input-249-cad3d6f51ff8>, line 1)

In [238]:
player_rows

[<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td>,
 <td class="posrela" title=""><table class="inline-table" title=""><tr><td class="" rowspan="2"><a href="#"><img alt="David von Ballmoos" class="bilderrahmen-fixed" src="https://tmssl.akamaized.net//images/portrait/small/203124-1533134983.jpg?lm=1533134993" title="David von Ballmoos"/></a></td><td class="hauptlink"><div class="di nowrap"><span class="hide-for-small"><a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">David von Ballmoos</a></span></div><div class="di nowrap"><span class="show-for-small"><a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">D. von Ballmoos</a></span></div></td></tr><tr><td>Goalkeeper</td></tr></table></td>,
 <td class="" rowspan="2"><a href="#"><img alt="David von Ballmoos" class="bilderrahmen-fixed" src="https://tmssl.ak

In [ ]:
player_row_items = player_

In [326]:
player_df = pd.DataFrame()
for player_row in player_rows[1:]:
    #parse inline_table    
    player_tags = player_row.find_all("td")
    
    player_number = player_tags[0].text
    player_name = player_tags[1].div.text #better a?
    player_link = player_tags[1].div.span.a['href']
    player_id = player_tags[1].div.span.a['id']
    try:
        player_smallimg = player_tags[2].a.img['src']
    except:
        continue
    player_birthday = player_tags[5].text
    player_country = player_tags[6].img["title"]
    player_value = player_tags[8].text

    player_dict = {
        #'player_tags'     :player_tags
        'player_number'  :player_number
        ,'player_name'    :player_name
        ,'player_link'    :player_link
        ,'player_id'      :player_id
        ,'player_smallimg':player_smallimg
        ,'player_birthday':player_birthday
        ,'player_country' :player_country
        ,'player_value'   :player_value
    }
    #for player_tag in player_tags:
    #    print(player_tag[])
    player_df = player_df.append(player_dict, ignore_index=True)
        


IndexError: list index out of range

In [327]:
player_df

""


In [211]:
player_df

,player_birthday,player_country,player_id,player_link,player_name,player_number,player_smallimg,player_value
0,"Dec 30, 1994 (24)",Switzerland,203124,/david-von-ballmoos/profil/spieler/203124,David von Ballmoos,26,https://tmssl.akamaized.net//images/portrait/s...,"3,50 mil. €"


In [124]:
player_row[:][1].find_all("td")

[<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td>,
 <td class="posrela" title=""><table class="inline-table" title=""><tr><td class="" rowspan="2"><a href="#"><img alt="David von Ballmoos" class="bilderrahmen-fixed" src="https://tmssl.akamaized.net//images/portrait/small/203124-1533134983.jpg?lm=1533134993" title="David von Ballmoos"/></a></td><td class="hauptlink"><div class="di nowrap"><span class="hide-for-small"><a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">David von Ballmoos</a></span></div><div class="di nowrap"><span class="show-for-small"><a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">D. von Ballmoos</a></span></div></td></tr><tr><td>Goalkeeper</td></tr></table></td>,
 <td class="" rowspan="2"><a href="#"><img alt="David von Ballmoos" class="bilderrahmen-fixed" src="https://tmssl.ak

In [250]:

BASE_URL = "https://www.transfermarkt.co.uk"


class Team:
	def __init__( self, url, name, scraper):
		self.LeagueName = name
		soup = scraper(url)
		#reading player table and filtering for offensive players
		playerTable = soup.find("table", class_="items")
		players = playerTable.find_all("a", class_="spielprofil_tooltip")[::2]
		offensivePlayers = filter( Team.isStrikerOrWinger, players)
		offensivePlayersUrls = [BASE_URL + player["href"] for player in offensivePlayers]
		#self.PlayerData = [PlayerProfile( playerUrl, scraper) for playerUrl in offensivePlayersUrls]
		self.PlayersData = []
		for playerUrl in offensivePlayersUrls:
			try:
				NewPlayerProfile = PlayerProfile( playerUrl, scraper)
				NewPlayerProfile.PlayerData["current league"] = self.LeagueName
				self.PlayersData.append( NewPlayerProfile)
			except:
				continue


	@staticmethod
	def isStrikerOrWinger( player):
		position = player.find_next("tr").text.strip().lower()
		return "wing" in position or "centre-forward" in position

In [253]:
N_LEAGUES = 10 #keeping the top N leagues
LEAGUES_URL = "https://www.transfermarkt.co.uk/wettbewerbe/europa/wettbewerbe"
BASE_URL = "https://www.transfermarkt.co.uk"

DELAY_BETWEEN_QUERIES = 0 #min delay in seconds spacing http queries
class PageScraper():
    def __init__(self ):
        self.opener = urllib.request.build_opener()
        self.opener.addheaders = [('User-agent', 'Mozilla/5.0')]
        self.lastQuery = -float("inf")
    def readUrl( self, url_):
        presentTime = time() - self.lastQuery - DELAY_BETWEEN_QUERIES
        if presentTime < 0:
            sleep( abs(presentTime))
        inData = self.opener.open(url_)
        content = inData.read()
        self.lastQuery = time()
        return bs(content, "html.parser")
    def __call__( self, url_):
        return self.readUrl( url_)



scraper = PageScraper()
soup = scraper( LEAGUES_URL)
LeagueTables = soup.find("table", class_="items").find("tbody")
Leagues = LeagueTables.find_all("a", href=re.compile("wettbewerb/[A-Z]{2}1"), title=re.compile("\w"))
Leagues = Leagues[:N_LEAGUES]
LeagueUrlDic = { league.text : BASE_URL + league["href"] for league in Leagues}
LeaguesData = []
for leagueName, leagueUrl in LeagueUrlDic.items():
	print( "Scraping the %s..." %leagueName)
	LeaguesData.append( League( leagueName, leagueUrl, scraper))

#flattening all players information to pandas.DataFrame and exporting to csv
PlayerProfiles = [player.PlayerData for league in LeaguesData for team in league.TeamsData for player in team.PlayersData]
df = pd.DataFrame( PlayerProfiles)
df.to_csv("StrikerPerformance.csv", index=False)

TypeError: 'module' object is not callable

In [95]:
players[0]

'<tr>\n <th class="zentriert" id="yw1_c0">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">\n   #\n  </a>\n </th>\n <th id="yw1_c1">\n  <a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">\n   Player\n  </a>\n </th>\n <th class="zentriert" id="yw1_c2">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">\n   Date of birth / Age\n  </a>\n </th>\n <th class="zentriert" id="yw1_c3">\n  Nat.\n </th>\n <th class="zentriert" id="yw1_c4">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">\n   Contract expires\n  </a>\n </th>\n <th class="rechts" id="yw1_c5">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">\n   Market value\n  </a>\n </th>\n</tr>\n'

In [90]:
players[1].find("title")

In [ ]:
pla

In [82]:
len(table.find_all("tr"))

76

In [ ]:
rows = soup.find("table", border=1).find("tbody").find_all("tr")

for row in rows:
    cells = row.find_all("td")
    rn = cells[0].get_text()

In [79]:
print(table[2])

IndexError: list index out of range

In [72]:
for player in table[0]:
    print(player)



<thead>
<tr>
<th class="zentriert" id="yw1_c0"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">#</a></th><th id="yw1_c1"><a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">Player</a></th><th class="zentriert" id="yw1_c2"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">Date of birth / Age</a></th><th class="zentriert" id="yw1_c3">Nat.</th><th class="zentriert" id="yw1_c4"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">Contract expires</a></th><th class="rechts" id="yw1_c5"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">Market value</a></th></tr>
</thead>


<tbody>
<tr class="odd">
<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td><td class="posrela" title=""><table class="inline-table" title=""><tr><td class="" rowspan="2"

In [70]:
len(tables)

1

In [54]:
print(tables[1])

<table class="items">
<thead>
<tr>
<th class="zentriert" id="yw1_c0"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">#</a></th><th id="yw1_c1"><a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">Player</a></th><th class="zentriert" id="yw1_c2"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">Date of birth / Age</a></th><th class="zentriert" id="yw1_c3">Nat.</th><th class="zentriert" id="yw1_c4"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">Contract expires</a></th><th class="rechts" id="yw1_c5"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">Market value</a></th></tr>
</thead>
<tbody>
<tr class="odd">
<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td><td class="posrela" title=""><table class="inline-table" title=""><tr><td cl

In [50]:
for table in tables:
    print(table)



<tbody>
<tr>
<td>
                                                    Filter by season                                                </td>
<td>
<div class="inline-select">
<select class="chzn-select" data-placeholder="Filter by season" name="saison_id" tabindex="0">
<option selected="selected" value="2019">19/20</option>
<option value="2018">18/19</option>
<option value="2017">17/18</option>
<option value="2016">16/17</option>
<option value="2015">15/16</option>
<option value="2014">14/15</option>
<option value="2013">13/14</option>
<option value="2012">12/13</option>
<option value="2011">11/12</option>
<option value="2010">10/11</option>
<option value="2009">09/10</option>
<option value="2008">08/09</option>
<option value="2007">07/08</option>
<option value="2006">06/07</option>
<option value="2005">05/06</option>
<option value="2004">04/05</option>
<option value="2003">03/04</option>
<option value="2002">02/03</option>
<option value="2001">01/02</option>
<option value="2000">00/01

In [ ]:
#player_table = soup.find_all('responsive-table', class_='outer-text')

In [20]:
table = soup.find('table', attrs={'class':'lineItemsTable'})
#table_body = table.find('tbody')

In [29]:
print(soup.find_all('table'))

[]


In [33]:
soup

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr/><center>nginx</center>
</body>
</html>

In [3]:
import urllib.request  # we are going to need to generate a Request object
from bs4 import BeautifulSoup as soup


# here we define the headers for the request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

# this request object will integrate your URL and the headers defined above
req = urllib.request.Request(url=my_url, headers=headers)

# calling urlopen this way will automatically handle closing the request
with urllib.request.urlopen(req) as response:
    page_html = response.read()

In [5]:
df = pd.read_html(page_html)

In [11]:
df[3]

,0,1
0,NaN,Marco WölfliM. Wölfli
1,NaN,Goalkeeper
